### Spark Hw3: Youtube comments analysis 推荐给动手能强力的同学, laioffer student used only.
#### Write a Spark program to analyze the text data.
#### 选作项目 Publish your notebook and send the link to mike@laioffer.com

In this notebook, we have a dataset of user comments for youtube videos related to animals or pets. We will attempt to identify cat or dog owners based on these comments, find out the topics important to them, and then identify video creators with the most viewers that are cat or dog owners.

The dataset provided for this coding test are comments for videos related to animals and/or pets. The dataset is 240MB compressed; please download the file using this google drive link:
https://drive.google.com/file/d/1o3DsS3jN_t2Mw3TsV0i7ySRmh9kyYi1a/view?usp=sharing

 The dataset file is comma separated, with a header line defining the field names, listed here:
● creator_name. Name of the YouTube channel creator.
● userid. Integer identifier for the users commenting on the YouTube channels.
● comment. Text of the comments made by the users.

Please use a recent version of PySpark (version 2.2 or higher) to analyze the data. Do not use
any external libraries; just use the native methods from pyspark.sql and pyspark.ml. (Do not
use pyspark.mllib as this has been deprecated.) Keep your code clean and efficient, with
enough documentation so that the grader can easily follow your train of thought. Summarize
the key results from each step. Explain how to execute your code from a command line
interface.

Step 1: Identify Cat And Dog Owners
Find the users who are cat and/or dog owners.

Step 2: Build And Evaluate Classifiers
Build classifiers for the cat and dog owners and measure the performance of the classifiers.

Step 3: Classify All The Users
Apply the cat/dog classifiers to all the users in the dataset. Estimate the fraction of all users
who are cat/dog owners.

Step 4: Extract Insights About Cat And Dog Owners
Find topics important to cat and dog owners.

Step 5: Identify Creators With Cat And Dog Owners In The Audience
Find creators with the most cat and/or dog owners. Find creators with the highest statistically
significant percentages of cat and/or dog owners.

In [0]:
# link: https://drive.google.com/file/d/1o3DsS3jN_t2Mw3TsV0i7ySRmh9kyYi1a/view?usp=sharing
# need to install googledrivedownloader==0.4 on cluster beforehand
#from google_drive_downloader import GoogleDriveDownloader as gdd
#gdd.download_file_from_google_drive(file_id='1o3DsS3jN_t2Mw3TsV0i7ySRmh9kyYi1a', #dest_path='./../../dbfs/laioffer/spark_hw3/data/animal_comments.gz')



#### 0. Data Exploration and Cleaning

In [0]:
df_clean=spark.read.csv("/FileStore/tables/animals_comments.csv",inferSchema=True,header=True)
df_clean.show(10)

+--------------------+------+-------------------------------------+
|        creator_name|userid|                              comment|
+--------------------+------+-------------------------------------+
|        Doug The Pug|  87.0|                 I shared this to ...|
|        Doug The Pug|  87.0|                   Super cute  😀🐕🐶|
|         bulletproof| 530.0|                 stop saying get e...|
|       Meu Zoológico| 670.0|                 Tenho uma jiboia ...|
|              ojatro|1031.0|                 I wanna see what ...|
|     Tingle Triggers|1212.0|                 Well shit now Im ...|
|Hope For Paws - O...|1806.0|                 when I saw the en...|
|Hope For Paws - O...|2036.0|                 Holy crap. That i...|
|          Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
|       Brian Barczyk|2698.0|                 Call the teddy Larry|
+--------------------+------+-------------------------------------+
only showing top 10 rows



In [0]:
df_clean.count() 

Out[3]: 5820035

In [0]:
df_clean = df_clean.na.drop(subset=["comment"])
df_clean.count()

Out[4]: 5818984

In [0]:
df_clean.show()

+--------------------+------+-------------------------------------+
|        creator_name|userid|                              comment|
+--------------------+------+-------------------------------------+
|        Doug The Pug|  87.0|                 I shared this to ...|
|        Doug The Pug|  87.0|                   Super cute  😀🐕🐶|
|         bulletproof| 530.0|                 stop saying get e...|
|       Meu Zoológico| 670.0|                 Tenho uma jiboia ...|
|              ojatro|1031.0|                 I wanna see what ...|
|     Tingle Triggers|1212.0|                 Well shit now Im ...|
|Hope For Paws - O...|1806.0|                 when I saw the en...|
|Hope For Paws - O...|2036.0|                 Holy crap. That i...|
|          Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
|       Brian Barczyk|2698.0|                 Call the teddy Larry|
|            The Dodo|2702.0|                   😐🤔😓😢😭😭😭😭😟|
|Hope For Paws - O...|2911.0|                 That mother cat l...|
|Hope For

In [0]:
# find user with preference of dog and cat
from pyspark.sql.functions import when
from pyspark.sql.functions import col

# you can user your ways to extract the label

df_clean = df_clean.withColumn("label", \
                           (when(col("comment").like("%my dog%"), 1) \
                           .when(col("comment").like("%I have a dog%"), 1) \
                           .when(col("comment").like("%I have a puppy%"), 1) \
                           .when(col("comment").like("%I have a kitty%"), 1) \
                           .when(col("comment").like("%I have a cat%"), 1) \
                           .when(col("comment").like("%I have dog%"), 1) \
                           .when(col("comment").like("%I have cat%"), 1) \
                           .when(col("comment").like("%my cat%"), 1) \
                           .when(col("comment").like("%my puppy%"), 1) \
                           .when(col("comment").like("%my dogs%"), 1) \
                           .when(col("comment").like("%my cats%"), 1) \
                           .when(col("comment").like("%my pup%"), 1) \
                           .when(col("comment").like("%my kitties%"), 1) \
                           .when(col("comment").like("%my kitty%"), 1) \
                           .when(col("comment").like("%my puppies%"), 1) \
                           .otherwise(0)))

In [0]:
df_clean.show()

+--------------------+------+-------------------------------------+-----+
|        creator_name|userid|                              comment|label|
+--------------------+------+-------------------------------------+-----+
|        Doug The Pug|  87.0|                 I shared this to ...|    0|
|        Doug The Pug|  87.0|                   Super cute  😀🐕🐶|    0|
|         bulletproof| 530.0|                 stop saying get e...|    0|
|       Meu Zoológico| 670.0|                 Tenho uma jiboia ...|    0|
|              ojatro|1031.0|                 I wanna see what ...|    0|
|     Tingle Triggers|1212.0|                 Well shit now Im ...|    0|
|Hope For Paws - O...|1806.0|                 when I saw the en...|    0|
|Hope For Paws - O...|2036.0|                 Holy crap. That i...|    0|
|          Life Story|2637.0|武器はクエストで貰えるんじゃないん...|    0|
|       Brian Barczyk|2698.0|                 Call the teddy Larry|    0|
|            The Dodo|2702.0|                   😐🤔😓😢😭😭😭😭😟|

#### 1. Data preprocessing and Build the classifier

In [0]:
from pyspark.ml.feature import RegexTokenizer, Word2Vec
from pyspark.ml.classification import LogisticRegression

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="comment", outputCol="words", pattern="\\W")

word2Vec = Word2Vec(inputCol="words", outputCol="features")

In [0]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[regexTokenizer, word2Vec])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(df_clean)
dataset = pipelineFit.transform(df_clean)

In [0]:
dataset.show()

+--------------------+------+-------------------------------------+-----+--------------------+--------------------+
|        creator_name|userid|                              comment|label|               words|            features|
+--------------------+------+-------------------------------------+-----+--------------------+--------------------+
|        Doug The Pug|  87.0|                 I shared this to ...|    0|[i, shared, this,...|[0.04145702736621...|
|        Doug The Pug|  87.0|                   Super cute  😀🐕🐶|    0|       [super, cute]|[0.04192611575126...|
|         bulletproof| 530.0|                 stop saying get e...|    0|[stop, saying, ge...|[0.02303081532855...|
|       Meu Zoológico| 670.0|                 Tenho uma jiboia ...|    0|[tenho, uma, jibo...|[-0.2456732491652...|
|              ojatro|1031.0|                 I wanna see what ...|    0|[i, wanna, see, w...|[-0.0715718987313...|
|     Tingle Triggers|1212.0|                 Well shit now Im ...|    0|[w

In [0]:
(lable0_train,lable0_test)=dataset.filter(col('label')==1).randomSplit([0.7, 0.3],seed = 100)
(lable1_train, lable1_ex)=dataset.filter(col('label')==0).randomSplit([0.005, 0.995],seed = 100)
(lable1_test, lable1_ex2)=lable1_ex.randomSplit([0.002, 0.998],seed = 100)

In [0]:
trainingData = lable0_train.union(lable1_train)
testData=lable0_test.union(lable1_test)

In [0]:
print("Dataset Count: " + str(dataset.count()))
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Dataset Count: 5818984
Training Dataset Count: 57177
Test Dataset Count: 23873


##### LogisticRegression

##### Parameter Tuning and K-fold cross-validation

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label')
#regularization parameter preventing overfitting
paramGrid = ParamGridBuilder() \
    .addGrid(lr.maxIter, [1, 5, 10]) \
    .addGrid(lr.regParam, [0.01, 0.5, 2.0]) \
    .build()
evaluator = BinaryClassificationEvaluator()
crossval = CrossValidator(estimator = lr,
                          estimatorParamMaps = paramGrid,
                          evaluator = evaluator,
                          numFolds = 5)

cvModel = crossval.fit(trainingData)
bestModel = cvModel.bestModel
bestPred = bestModel.transform(testData)

/databricks/spark/python/pyspark/ml/util.py:838: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
  warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)


In [0]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.evaluation import MulticlassMetrics
bestPred = bestPred.withColumn("label",col("label").cast('double'))
L_P = bestPred.select("label", "prediction").rdd
# Instantiate metrics object
metrics = MulticlassMetrics(L_P)
precision = metrics.precision(label = 1.0)
recall = metrics.recall(label = 1.0)
f1Score = metrics.fMeasure(label = 1.0)


In [0]:
#print out the best parameters of the best model and evalution of the model
print("Best Param (regParam): ", bestModel._java_obj.getRegParam())
print("Best Param (MaxIter): ", bestModel._java_obj.getMaxIter())
print("Precision score:", precision)
print("Recall score:", recall)
print("F1 score:", f1Score)
print("Best Model Test Area Under ROC:", evaluator.evaluate(bestPred))

Best Param (regParam):  0.01
Best Param (MaxIter):  10
Precision score: 0.923912169227057
Recall score: 0.891155934833204
F1 score: 0.9072384788532164
Best Model Test Area Under ROC: 0.9537300700988066


##### RandomForest

In [0]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
#regularization parameter preventing overfitting
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [5, 10, 20]) \
    .addGrid(rf.maxDepth, [2, 5, 10]) \
    .build()
evaluator = BinaryClassificationEvaluator()
crossval = CrossValidator(estimator = rf,
                          estimatorParamMaps = paramGrid,
                          evaluator = evaluator,
                          numFolds = 5)

cvModel = crossval.fit(trainingData)
bestModel = cvModel.bestModel
bestPred = bestModel.transform(testData)

/databricks/spark/python/pyspark/ml/util.py:838: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
  warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)


In [0]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.evaluation import MulticlassMetrics
bestPred = bestPred.withColumn("label",col("label").cast('double'))
L_P = bestPred.select("label", "prediction").rdd
# Instantiate metrics object
metrics = MulticlassMetrics(L_P)
precision = metrics.precision(label = 1.0)
recall = metrics.recall(label = 1.0)
f1Score = metrics.fMeasure(label = 1.0)

In [0]:
#print out the best parameters of the best model and evalution of the model
#print("Best Param (numTrees): ", bestModel._java_obj.getnumTrees())
#print("Best Param (maxDepth): ", bestModel._java_obj.getmaxDepth())
print("Precision score:", precision)
print("Recall score:", recall)
print("F1 score:", f1Score)
print("Best Model Test Area Under ROC:", evaluator.evaluate(bestPred))

Precision score: 0.9416874447036113
Recall score: 0.8864324651726226
F1 score: 0.9132249132249132
Best Model Test Area Under ROC: 0.9629099736827158


##### Gradient boosting

In [0]:
#build gradient boost tree classifier
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(featuresCol = 'features', labelCol = 'label')
paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [2, 3, 5]) \
    .build()
evaluator = BinaryClassificationEvaluator()
crossval = CrossValidator(estimator = gbt,
                          estimatorParamMaps = paramGrid,
                          evaluator = evaluator,
                          numFolds = 5)

cvModel = crossval.fit(trainingData)
bestModel = cvModel.bestModel
bestPred = bestModel.transform(testData)

In [0]:
bestPred = bestPred.withColumn("label",col("label").cast('double'))
L_P = bestPred.select("label", "prediction").rdd
# Instantiate metrics object
metrics = MulticlassMetrics(L_P)
precision = metrics.precision(label = 1.0)
recall = metrics.recall(label = 1.0)
f1Score = metrics.fMeasure(label = 1.0)

In [0]:
#evalution of the model
#print("Best Param (maxDepth): ", bestModel._java_obj.getmaxDepth())
#print("Best Param (MaxIter): ", bestModel._java_obj.getMaxIter())
print("Precision score:", precision)
print("Recall score:", recall)
print("F1 score:", f1Score)
print("Best Model Test Area Under ROC:", evaluator.evaluate(bestPred))

Precision score: 0.9140899078077833
Recall score: 0.8873049813890869
F1 score: 0.9004983121684617
Best Model Test Area Under ROC: 0.9598052230991487


#### 2. Classify All The Users using Logistic Regression model
##### After comparing the recall value, random forest model has the best performance

In [0]:
total_user = dataset.select('comment').count()
#pred_owner = bestPred.filter(col('prediction') == 1).select('userid').distinct().count()
pred_owner_num = bestPred.filter("prediction = 1.0").count()

In [0]:
proportion = pred_owner_num / total_user
print(proportion)

0.0022151633343552757


#### 3. Get insigts of Users

In [0]:
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import explode
from pyspark.sql import SparkSession
pred_owner = bestPred.filter("prediction = 1.0")
words_explode = pred_owner.select(pred_owner.userid,explode(pred_owner.words))
words_count = words_explode.groupBy("col").count()
words_count = words_count.orderBy(col("count").desc())
freq_words = words_count.rdd.map(lambda r: r.col).collect()
print(freq_words)

['my', 'i', 'and', 'to', 'the', 'a', 'dog', 'is', 'cat', 'of', 'it', 'he', 'that', 'was', 'you', 'in', 'she', 'me', 'have', 'so', 'but', 'for', 'like', 'her', 'with', 'this', 'when', 'on', 'dogs', 'him', 'just', 'they', 'if', 'your', 'cats', 'do', 'one', 'not', 'be', 'are', 'we', 'them', 'love', 'at', 'would', 'had', 'get', 'im', 'all', 'his', 'as', 'or', 'can', 'because', 'up', 'its', 'out', 'what', 'dont', 'from', 'how', 'will', 'got', 'know', 'has', 'now', 'about', 'time', 'puppy', 'much', 'there', 'really', 'looks', 'then', 'too', 'want', 'go', 'even', 'same', 'old', 'day', 'other', 'only', 'very', 'some', 'never', 'also', 'always', 'people', 'think', 'no', 'could', 'hes', 'does', 'video', 'back', 'help', 'who', 'see', 'died', 'were', '2', 'by', 'good', 'been', 'away', 'an', 'years', 'more', 'food', 'their', 'still', 'am', 'lol', 'take', 'did', 'cute', 'after', 'put', 'make', 'well', '3', 'videos', 'shes', 'down', 'give', 'little', 'any', 'home', 'cant', 'thing', 'going', 'way', 'n

#### 4. Identify Creators With Cat And Dog Owners In The Audience

In [0]:
bestPred.registerTempTable("bestPred")

In [0]:
%sql SELECT creator_name, count(label) FROM bestPred WHERE label = 1.0 GROUP BY creator_name ORDER BY count(label) DESC

creator_name,count(label)
The Dodo,1221
Cole & Marmalade,868
Gohan The Husky,712
Zak Georges Dog Training rEvolution,697
Hope For Paws - Official Rescue Channel,623
Gone to the Snow Dogs,555
Vet Ranch,545
Brian Barczyk,513
Robin Seplut,486
Taylor Nicole Dean,480


#### 5. Analysis and Future work

From our analysis, we can see that the proportion of pet owners I selected from the population is 0.2%. After taking a look at their frequently used words, I think that they might be interested in topics like "videos", "age", "foods", "family","lost","train", "friends" and etc. I would consider promoting related videos or commercials to these pets owners. In addition, "The Dodo", "Cole & Marmalade" and "Gohan The Husky" are the three video creators with the most comments from pet owners. I would consider collaborating with them to do advertisements for pet related products. One problem of my subject is that the proportion of pet owners I calculated is 0.2% which is far lower than the US pet owning rate(about 40%). To solve this, I need to improve the way I label the pet owners through thinking about more comments that could represent the pet owners.

From project to your CV format 
1.overview of project 
2.data clean and modeling 
3.data analysis 
4.build ml model
5.recommendation based on the model results

1. I used a dataset of user comments for youtube videos related to animals or pets to identify cat or dog owners based on their comments.In addition, I find out the topics important to them, and then identify video creators with the most viewers that are cat or dog owners.
2. The dataset was cleaned by deleting all the rows containing null values and users who definitely own cats or dogs were labelled. Then the "word2vec" package was used to turn the comments into vectors and were used as features in later analysis
3. Logistic regression, random forest and GDBT were used to build the model while logistic regression has the best performance with highest recall value.
4. After selecting pet owners, I identified the most frequent words in their comments to predict the potential topics they are interested in and also found out video creators with most comments from pet owners. These video creators would be considered as potential people to collaborate with to promote our pet-related products.